#### Import libraries

In [340]:
import json
import math
import os
import shutil
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import time

##### Initialize Interface

In [341]:
filename = 'information.json'
root = Tk()
frm = ttk.Frame(root, padding=10)
frm.grid()
app_width = 500
app_height = 500
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width / 2) - (app_width / 2)
y = (screen_height / 2) - (app_height / 2)
root.geometry(f'{app_width}x{app_height}+{int(x)}+{int(y)}')

''

##### Initialize files

In [342]:
def getDir(library_path):
    if library_path and os.path.isdir(library_path):
        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]
        
        # Save folders to a file
        with open('avaliable_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    else:
        print("ERROR: no directory listed!!! Add it to the information.json file manually...")

In [343]:
if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "Nu",
            "PC Capacity": "Nu",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)

# Read data from json file
with open(filename, 'r') as json_file:
    data = json.load(json_file)

# There is a directory that the user has entered
if os.path.isdir(data["PC Directory"]):
    getDir(data["PC Directory"])


# TODO
# The user has not updated the directory to read from
#else:
    # Gray out the button for transfering anime.
    # Gray out the PC capacity
    # Gray out the limit for pc limit
    # gray out anime to transfer
    # Gray out the get list of avaliable anime button
    # Gray out the transfer button.
    # Pretty much wait until the user has chosen a valid directory

#### Functions

##### Get size of directory

In [344]:
def get_directory_size(directory):
    total_size = 0
    # Walk through each folder, subfolder, and files
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # Get the full file path
            file_path = os.path.join(dirpath, filename)
            # Add the file size to the total size
            if os.path.isfile(file_path):
                total_size += os.path.getsize(file_path)
                
    if total_size == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(math.floor(math.log(total_size, 1024)))
    p = math.pow(1024, i)
    s = round(total_size / p, 2)
    return f"{s} {size_name[i]}"

##### Progress bar

In [345]:
def senseStorage(src_dir, dst_dir, root2):
    # Sense how much the size of the files is to be transfered
    
    total = float(str.split(get_directory_size(src_dir))[0])
    # Instantiate the progress bar
    my_progress = ttk.Progressbar(root2, orient=HORIZONTAL, length=270, mode='determinate')
    my_progress.grid()
    
    progress = 0
    print(f"Total: {total}")
    # Periodically check the size of the files within the src_dir and update the progress variable
    shutil.move(src_dir, dst_dir)
    while progress != total:
        # Get the progress
        progress = total - int(str.strip(get_directory_size(src_dir))[0])
        print(f"updated progress: {progress}")
        perc_prog = (progress / total) * 100
        print(f"percent progress: {perc_prog}")
        # Update the progress bar variable
        my_progress['value'] = perc_prog

##### Function to retrieve files

In [346]:
def retrieve(titles, data, root2):
    # For each title in the list of shows
    for title in titles:
        
        # Get directory of show folder
        src_dir = os.path.join(data["PC Directory"], title).replace("/", "\\")
        
        # Destination folder
        dest_dir = os.path.join(data["Phone Directory"], title).replace("/", "\\")
        
        # Phone directory
        phone_dir = data["Phone Directory"]
        
        # Check if the show folder exists
        if not os.path.isdir(src_dir):
            print(f"ERROR: Source folder '{src_dir}' does not exist!")
            continue
        
        # Check if the destination folder already exists
        if not os.path.isdir(dest_dir) and os.path.isdir(phone_dir):
            
            senseStorage(src_dir, data["Phone Directory"], root2)
            
            print(f"Transferred '{title}' to '{phone_dir}'.")
        else:
            print(f"ERROR: Destination path '{dest_dir}' already exists!!!")

##### Function to store files

In [347]:
def store(titles, data, root2):
    
    # For each title in the list of shows
    for title in titles:
        
        # Get directory of show folder
        src_dir = os.path.join(data["Phone Directory"], title).replace("/", "\\")
        
        # Destination folder
        dest_dir = os.path.join(data["PC Directory"], title).replace("/", "\\")
        
        # Phone directory
        pc_dir = data["PC Directory"]
        
        # Check if the show folder exists
        if not os.path.isdir(src_dir):
            print(f"ERROR: Source folder '{src_dir}' does not exist!")
            continue
        
        # Check if the destination folder already exists
        if not os.path.isdir(dest_dir) and os.path.isdir(pc_dir):
            
            # Move folder and its contents to the new folder
            senseStorage(src_dir, pc_dir, root2)
            
            print(f"Transferred '{title}' to '{pc_dir}'.")
        else:
            print(f"ERROR: Destination path '{dest_dir}' already exists!!!")

#### Buttons

##### Set PC Directory

In [348]:
def setPCDir():
    if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "Nu",
            "PC Capacity": "Nu",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
    folder_path = filedialog.askdirectory()
    if folder_path:
        ttk.Label(frm, text=folder_path).grid(column=1, row=1)
        # Read the existing JSON data to update it
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        
        # Update the relevant fields in the data dictionary
        data["PC Directory"] = folder_path
        
        # Write the updated data back to the JSON file
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"{filename} has been updated with the new PC Directory.")

##### Set Phone Directory

In [349]:
def setPhoneDir():
    if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "Nu",
            "PC Capacity": "Nu",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
    folder_path = filedialog.askdirectory()
    if folder_path:
        ttk.Label(frm, text=folder_path).grid(column=1, row=2)
        # Read the existing JSON data to update it
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        
        # Update the relevant fields in the data dictionary
        data["Phone Directory"] = folder_path
        
        # Write the updated data back to the JSON file
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"{filename} has been updated with the new Phone Directory.")

##### Transfer Anime

In [350]:
def transferAnime():
    # Run a new instance of ttk
    root2 = Tk()
    frm2 = ttk.Frame(root2, padding=10)
    frm2.grid()
    app_width = 500
    app_height = 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width / 2) - (app_width / 2)
    y = (screen_height / 2) - (app_height / 2)
    root2.geometry(f'{app_width}x{app_height}+{int(x + 100)}+{int(y + 100)}')
    
    # Get a list of the anime inside of the folder
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    if data:
        # Get the directory of the library
        library_path = data["PC Directory"]

        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]

        # Save folders to a file
        with open('avaliable_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    
    # Instantiate and grid scrollbar and listbox
    my_scrollbar = Scrollbar(frm2, orient=VERTICAL)
    my_listbox = Listbox(frm2, width=50, yscrollcommand=my_scrollbar.set, selectmode=MULTIPLE)
    my_scrollbar.config(command=my_listbox.yview)
    frm2.grid()
    my_listbox.grid(pady=15)
    my_listbox.grid(row=0, column=0)
    my_scrollbar.grid(row=0, column=1, sticky='ns')
    
    # Add shows to listbox from avaliable anime
    shows = []
    with open('avaliable_anime.txt', 'r') as file:
        for line in file:
            shows.append(line)
    for show in shows:
        my_listbox.insert(0, show)
    
    
    # Create an entry box
    my_entry = Entry(root2)
    my_entry.grid()
    
    # Update entrybox with listbox clicked
    def fillout(e):
        selection = my_listbox.curselection()
        
        # Delete whatever is in the entrybox
        my_entry.delete(0, END)
        
        # Add clicked list item to entry box (should I sub this out with anchor?????)
        my_entry.insert(0, my_listbox.get(selection[-1]))
        
    # Update search bar with entry from listbox
    my_listbox.bind("<<ListboxSelect>>", fillout)
    
    
    def update(data):
        
        my_listbox.delete(0, END)
        
        # Add toppings to listbox
        for show in data:
            my_listbox.insert(END, show)
    
    
    
    # Checks entry vs listbox
    def check(e):
        
        # Grab what was typed
        typed = my_entry.get()
        
        if typed == '':
            data = shows
        else:
            data = []
            for show in shows:
                if typed.lower() in show.lower():
                    data.append(show)
                    
        # Update our listbox with selected items
        update(data)
        # See if the string is within the listbox item list
    my_entry.bind("<KeyRelease>", check)
    
    # Select All Button to show entries as labels
    def transferSelected():
        test = []
        for item in my_listbox.curselection():
            test.append(str.strip((my_listbox.get(item)))       )
        combined_text = ', '.join(test)
        combined_label = Label(root2, text=combined_text)
        combined_label.grid()  # Place it below the individual labels
        # Function to transfer the items selected
        retrieve(test, data, root2)
        for item in reversed(my_listbox.curselection()):
            my_listbox.delete(item)

    # Transfer Selected titles
    transferSelected_button = Button(root2, text="Export Selected", command=transferSelected)
    transferSelected_button.grid(pady=10)
    
    
    my_label = Label(root2, text='')
    my_label.grid(pady=5)

##### Store Anime

In [351]:
def storeAnime():
    # Run a new instance of ttk
    root2 = Tk()
    frm2 = ttk.Frame(root2, padding=10)
    frm2.grid()
    app_width = 500
    app_height = 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width / 2) - (app_width / 2)
    y = (screen_height / 2) - (app_height / 2)
    root2.geometry(f'{app_width}x{app_height}+{int(x + 100)}+{int(y + 100)}')
    # Get a list of the anime inside of the folder
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    if data:
        # Get the directory of the library
        library_path = data["Phone Directory"]

        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]

        # Save folders to a file
        with open('phone_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    
    # Instantiate and grid scrollbar and listbox
    my_scrollbar = Scrollbar(frm2, orient=VERTICAL)
    my_listbox = Listbox(frm2, width=50, yscrollcommand=my_scrollbar.set, selectmode=MULTIPLE)
    my_scrollbar.config(command=my_listbox.yview)
    frm2.grid()
    my_listbox.grid(pady=15)
    my_listbox.grid(row=0, column=0)
    my_scrollbar.grid(row=0, column=1, sticky='ns')
    
    # Add shows to listbox from avaliable anime
    shows = []
    with open('phone_anime.txt', 'r') as file:
        for line in file:
            shows.append(line)
    for show in shows:
        my_listbox.insert(0, show)
    
    
    # Create an entry box
    my_entry = Entry(root2)
    my_entry.grid()
    
    # Update entrybox with listbox clicked
    def fillout(e):
        selection = my_listbox.curselection()
        
        # Delete whatever is in the entrybox
        my_entry.delete(0, END)
        
        # Add clicked list item to entry box (should I sub this out with anchor?????)
        my_entry.insert(0, my_listbox.get(selection[-1]))
        
    # Update search bar with entry from listbox
    my_listbox.bind("<<ListboxSelect>>", fillout)
    
    
    def update(data):
        
        my_listbox.delete(0, END)
        
        # Add toppings to listbox
        for show in data:
            my_listbox.insert(END, show)
    
    # Checks entry vs listbox
    def check(e):
        
        # Grab what was typed
        typed = my_entry.get()
        
        if typed == '':
            data = shows
        else:
            data = []
            for show in shows:
                if typed.lower() in show.lower():
                    data.append(show)
                    
        # Update our listbox with selected items
        update(data)
        # See if the string is within the listbox item list
    my_entry.bind("<KeyRelease>", check)
    
    # Select All Button to show entries as labels
    def transferSelected():
        test = []
        for item in my_listbox.curselection():
            test.append(str.strip((my_listbox.get(item))))
        combined_text = ', '.join(test)
        combined_label = Label(root2, text=combined_text)
        combined_label.grid()
        # Function to transfer the items selected
        store(test, data, root2)
        for item in reversed(my_listbox.curselection()):
            my_listbox.delete(item)

    # Transfer Selected titles
    transferSelected_button = Button(root2, text="Export Selected", command=transferSelected)
    transferSelected_button.grid(pady=10)
    
    
    my_label = Label(root2, text='')
    my_label.grid(pady=5)

#### Progress bars

##### Progress Bars for Storing Anime

##### Progress Bars for Retrieving Anime

#### Rows

##### Row 0

In [352]:
ttk.Label(frm, text="Welcome to the anime library!").grid(column=0, row=0)

##### Row 1

In [353]:
ttk.Label(frm, text="PC anime directory:").grid(column=0, row=1)
ttk.Button(frm, text="..", command=setPCDir).grid(column=2, row=1)

##### Row 2

In [354]:
ttk.Label(frm, text="Phone anime directory:").grid(column=0, row=2)
ttk.Button(frm, text="..", command=setPhoneDir).grid(column=2, row=2)


##### Row 3

In [355]:
# Get the storage usage on pc
if os.path.isdir(data["PC Directory"]):
    total = get_directory_size(data["PC Directory"])
total = total if total else "unknown"
ttk.Label(frm, text=f"PC storage capacity: {total}/TOTAL GB").grid(column=0, row=3)

# TODO:
    # Currently the storage usage is inaccurate
    # How to update the pc usage when the directory is reset

##### Row 4

In [356]:
if os.path.isdir(data["Phone Directory"]):
    total = get_directory_size(data["Phone Directory"])
ttk.Label(frm, text=f"Phone storage capacity: {total}/TOTAL GB").grid(column=0, row=4)


##### Row 5

In [357]:
ttk.Button(frm, text="Retrieve", command=transferAnime).grid(column=0, row=5)
ttk.Button(frm, text="Store", command=storeAnime).grid(column=1, row=5)

##### Row 8

In [358]:
ttk.Button(frm, text="Quit", command=root.destroy).grid()
root.mainloop()